# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import geoviews as gv
import requests
import folium

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port elizabeth,-33.9180,25.5701,294.44,97,75,2.57,ZA,1710118662
1,1,port-aux-francais,-49.3500,70.2167,280.90,94,100,16.45,TF,1710118662
2,2,nikolayevsk-on-amure,53.1406,140.7300,266.58,81,68,2.02,RU,1710118662
3,3,marsh harbour,26.5412,-77.0636,296.27,80,100,7.53,BS,1710118662
4,4,albany,42.6001,-73.9662,274.45,72,97,3.58,US,1710118489


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Create a map centered at a location of your choice
map_city = folium.Map(location=[0, 0], zoom_start=2)

# Iterate over each row in the DataFrame
for index, row in city_data_df.iterrows():
    # Add a circle marker for each city, with radius proportional to humidity
    folium.CircleMarker(location=[row['Lat'], row['Lng']], 
                        radius=row['Humidity']/20,  
                        popup=row['City'], 
                        color='blue', 
                        fill=True, 
                        fill_color='blue').add_to(map_city)

# Display the map
map_city

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Define the new ideal weather conditions
ideal_conditions = {
    'Max Temp': (0, 100),  # Temperature range in Celsius
    'Wind Speed': (0, 20),  # Wind speed in m/s
    'Cloudiness': 100 # Cloudiness percentage
}

# Filter cities that meet the new ideal weather conditions using query
ideal_cities_df = city_data_df.query(
    f"`Max Temp` > {ideal_conditions['Max Temp'][0]} and `Max Temp` < {ideal_conditions['Max Temp'][1]} and "
    f"`Wind Speed` < {ideal_conditions['Wind Speed'][1]} and `Cloudiness` <= {ideal_conditions['Cloudiness']}"
)

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df = hotel_df.assign(Hotel_Name="")

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel_Name
0,port elizabeth,ZA,-33.9180,25.5701,97,
1,port-aux-francais,TF,-49.3500,70.2167,94,
2,nikolayevsk-on-amure,RU,53.1406,140.7300,81,
3,marsh harbour,BS,26.5412,-77.0636,80,
4,albany,US,42.6001,-73.9662,72,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:

city_counter = 0
for index, row in city_data_df.iterrows():
    # Extract city-specific information
    latitude = row['Lat']
    longitude = row['Lng']
    city = row['City']
    country = row['Country']
    humidity = row['Humidity']
    
    # Set the parameters for the type of place
    categories = "accommodation.hotel"
    radius = 1000

    # Set the parameters for the type of search
    filter_param = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"
    limit = 20

    # Set up a parameters dictionary
    params = {
        "categories": categories,
        "limit": limit,
        "filter": filter_param,
        "bias": bias,
        "apiKey": geoapify_key  # Replace "geoapify_key" with your actual Geoapify API key
    }

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Run a request using our params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    data = response.json()
    try:
        hotel_name = data['features'][0]['properties']['name']
        country_code = data['features'][0]['properties']['country_code']
        hotel_formatted_name = f"{hotel_name}, {city}, {country_code}"
    except (KeyError, IndexError):
        hotel_formatted_name = "No hotel found"

    # Assign the hotel name to the 'Hotel_Name' column in the hotel_df DataFrame
    hotel_df.loc[index, 'Hotel_Name'] = hotel_formatted_name
    
    # Print the city and corresponding hotel name
    print(f"For {city}, {country}: {hotel_formatted_name}")
    print(data)
    print()  # Add a line break for clarity
    # Increment the city counter
    city_counter += 1
    
    # Check if the counter exceeds 60
    if city_counter >= 60:
        break  # Exit the loop if 60 cities have been processed

For port elizabeth, ZA: No hotel found
{'type': 'FeatureCollection', 'features': []}

For port-aux-francais, TF: Keravel, port-aux-francais, fr
{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'Keravel', 'country': 'France', 'country_code': 'fr', 'state': 'French Southern and Antarctic Lands', 'city': 'Port-aux-Français', 'municipality': 'Kerguelen Archipelago', 'street': 'Chemin de Keravel', 'lon': 70.21991619089022, 'lat': -49.34887615, 'formatted': 'Chemin de Keravel, Port-aux-Français, France', 'address_line1': 'Chemin de Keravel', 'address_line2': 'Port-aux-Français, France', 'categories': ['accommodation', 'accommodation.hotel', 'building', 'building.accommodation'], 'details': [], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database Licence', 'url': 'https://www.openstreetmap.org/copyright', 'raw': {'name': 'Keravel', 'osm_id': 187212461, 'tourism': 'hotel', 'building': 'yes', 

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
# Create a map centered at a location of your choice
map_city = folium.Map(location=[0, 0], zoom_start=2)

# Iterate over each row in the DataFrame
for index, row in hotel_df.iterrows():
    # Construct popup text with city, hotel name, and country
    popup_text = f"Hotel: {row['Hotel_Name']}<br>City: {row['City']}<br>Country: {row['Country']}"
    
    # Add a circle marker for each city, with radius proportional to humidity
    folium.CircleMarker(location=[row['Lat'], row['Lng']], 
                        radius=row['Humidity']/20,  
                        popup=popup_text,  
                        color='blue', 
                        fill=True, 
                        fill_color='blue').add_to(map_city)

# Display the map
map_city
